In [1]:
import os, sys
from pathlib import Path
PROJECTPATH = Path(".").cwd().parent.absolute()
sys.path.insert(0, PROJECTPATH)
os.chdir(PROJECTPATH)
PROJECTPATH

PosixPath('/home/jmgibso3/ncstate/projects/auth_checker')

In [2]:
from auth_checker.authz.authorizer import Authorizer

In [3]:
authz = Authorizer()
e = authz.enforcer

In [4]:
apps = [
    "clearance",
    "liaison",
    "userlookup",
    "automations",
    "proxies",
]

roles = {
    "staff": "rw",
    "admin": "app",
    "dev": "root",
}


# With group associations the 1 position becomes implicit to the 0 position
# Thus admin can do all that staff can do, but staff is not permitted admin
# Anything admin can do dev can do, but admin will not have dev permissions
group_associations = [
    ('admin', 'staff'),
    ('dev', 'admin')
]


dev_users = [
    'jmgibso3@ncsu.edu',
    'staylor8@ncsu.edu',
    'jtchampi@ncsu.edu',
    'rsemmle@ncsu.edu',
    'lgsanche@ncsu.edu',
    'ewbrenna@ncsu.edu'
]

In [63]:
def set_policies():
    for app in apps:
        for role, perm in roles.items():
            # Namespaced for finer grained control outside SAT
            e.add_policy(f"{role}:{app}", app, perm)
            # Groupish
            e.add_policy(role, app, perm)

def set_group_associations():
    for assoc in group_associations:
        e.add_role_for_user(*assoc)

def set_dev_users():
    for user in dev_users:
        e.add_role_for_user(user, 'dev')

def clear_policy_store():
    for role in e.get_all_roles():
        e.delete_role(role)
    for subj in e.get_all_subjects():
        e.delete_user(subj)

In [6]:
authz.roles_for_user('jmgibso3@ncsu.edu')

['dev']

In [60]:
set_policies()

In [62]:
set_group_associations()

In [58]:
clear_policy_store()

In [64]:
set_dev_users()

In [72]:
'admin' in e.get_implicit_roles_for_user('jmgibso3@ncsu.edu')

True

In [66]:
e.enforce('jtchampi@ncsu.edu', 'clearance', 'rw')

True

In [67]:
e.get_roles_for_user('jtchampi@ncsu.edu')

['dev']

In [68]:
e.get_implicit_roles_for_user('jtchampi@ncsu.edu')

['dev', 'admin', 'staff']